### 作业1.基于 data 目录下的数据训练 ChatGLM3 模型，使用 inference Notebook 对比微调前后的效果。

现将160个样本的训练集使用qlora微调了一版ChatGLM3模型，其中epoch为3，最终训练损失为0.42。

1.加载原始模型

In [7]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [8]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Loading checkpoint shards: 100%|██████████████████| 7/7 [00:04<00:00,  1.48it/s]


In [10]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                         revision='b098244')

In [12]:
# 进行推断
input_text = "解释下乾卦是什么？"
response, history = base_model.chat(tokenizer, query=input_text)
print(response)

乾卦是八卦之一，也是八宫图中的第一个卦。乾卦是由两个阴爻夹一个阳爻构成，象征着天、云和雷等自然现象。乾卦的含义包括力量、刚强、创造、领导等。在八宫图中，乾卦位于北方，与事业、努力、坚定等有关。在命理学中，乾卦也与人君、权力、法律等有关。


In [13]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，也是八宫图中的第二个卦。讼卦由三个阴爻夹一个阳爻构成，象征着诉讼、争端、法律等。坎坷、艰难、矛盾、困苦等也是讼卦的象征。在八宫图中，讼卦位于西北，与法律、争端、权力等有关。在命理学中，讼卦也与人运、运势、健康等有关。


2.加载微调后的 ChatGLM3-6B

In [15]:
from peft import PeftModel, PeftConfig

peft_model_path = f"models/THUDM/chatglm3-6b-epoch3"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))"

In [16]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

In [20]:
# 微调前后效果对比
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model, training_tag)

问题：地水师卦是什么？

原始输出：
在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。

根据《象辞》，师卦由坤卦坎卦组成，象征着“地中有水”。君子观此卦象，取法于容纳江河的大地，收容和畜养大众，展示了一种包容和辅助的品质。师卦的主题是围绕着众人的困境和劳苦，表现出排除万难和公正无私的态度。其中，“丈人吉，无咎”表明只有德高望重的长者能够率领军队才能吉祥无咎。这对于处在困境中的人来说，也是一个重要的提醒，需要包容别人，艰苦努力，摒除一切困难。

在传统解卦中，师卦代表养兵聚众，出师攻伐的象征，但在这种情况下，彼此都会受伤，难得安宁。因此，需要谨慎行事，避免盲目妄动，保全自己，保持冷静机动，严于律已。在事业、经商、求名、婚恋、决策等方面都需要警惕潜在的敌人，谨小慎微，行为果断，从容沉着对付一切。只有如此，才能成功化解困境，获得名利双全。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))）：
[gMASK]sop 地水师卦是什么？ 在周易中，地水师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“地水师”，即军队，寓意着兵力和资源的运输、调度以及作战指挥。

地水师卦的核心哲学是： index>

- 君子观此卦象，从而思辨处理事态的经验和哲学
- 君子观此卦象，从而了解域外敌国的情况

在占卜中，地水师卦代表君子，象征着事态的发展。象征着冰（坎）和众（坤）相交，即地水和水的交合，暗示着理解和包容他人，等待时机，耐心等待，不可强求。

地水师卦象征着事态的发展和变化，提示着君子应该等待时机，以柔克刚，等待机会，不可强求。在经商、求名、婚恋、决策等方面，都需要等待合

In [19]:
base_response, ft_response = compare_chatglm_results("讼卦的哲学思想", base_model, qlora_model, training_tag)

问题：讼卦的哲学思想

原始输出：
在周易中，讼卦是由乾上坎下，刚与险相对的卦象。它预示着虽有利可图，但需要警惕戒惧。事情在中间阶段吉利，但最终会带来凶险。因此，对于涉及大川的行动不利，但会见大人却会有利。讼卦象征着天与水违行，即乾天升而坎水降，这种流向相背的情况，象征着事理的乖舛和无法调和。因此，君子观此卦象，以杜绝争讼为意，从而在谋事之初必须慎之又慎。讼卦的解释中指出，身心不安，事多不顺，宜修身养性，谨慎处事。在运势上，事与愿违，凡事不顺，宜防陷阱。在事业上，起初顺利，有利可图，继而受挫，务必警惕，慎之又慎，不得固执已见。在决策上，争强好胜，不安于现状，为改变命运和超越他人而奋斗。头脑聪颖，反应敏捷，有贵人相助。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，讼卦提示着需要接受教训，知足适可而止，可以功成名就。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))）：
[gMASK]sop 讼卦的哲学思想与解卦

 在周易中，讼卦是由乾上坎下，刚与险相对的卦象。它预示着虽有利可图，但需要警惕戒惧。事情在中间阶段吉利，但最终会带来凶险。因此，对于涉及大川的行动不利，但会见大人却会有利。讼卦象征着天与水违行，即乾天升而坎水降，这种流向相背的情况，象征着事理的乖舛和无法调和。因此，君子观此卦象，以杜绝争讼为意，从而在谋事之初必须慎之又慎。讼卦的解释中指出，身心不安，事多不顺，宜修身养性，谨慎处事。在运势上，事与愿违，凡事不顺，宜防陷阱。在事业上，起初顺利，有利可图，继而受挫，务必警惕，慎之又慎，不得固执已见。在决策上，争强好胜，不安于现状，为改变命运和超越他人而奋斗。头脑聪颖，反应敏捷，有贵人相助。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，讼卦提示着需要接受教训，知足适可而止，可以功成名就。


**结论：模型微调前后推断效果变化不大，没有明显的提升。**